In [1]:
import pandas as pd

path_to_train = './data/train.csv'
path_to_test = './data/test.csv'

categorical_features = ['listen_type', 
                        'genre_id', 
                        'album_id', 
                        'context_type',
                        'platform_name',
                        'artist_id']

train = pd.read_csv(path_to_train, encoding='utf-8', usecols=['user_id', 'ts_listen', 'is_listened'] + categorical_features)
test = pd.read_csv(path_to_test, encoding='utf-8', usecols=['user_id', 'ts_listen'] + categorical_features)
test['is_listened'] = 0 # просто щоб зібрати, ми будемо використовувати попередні значення

path_to_test_ts_listen = './data/features/ts_listen.test.csv'
path_to_train_ts_listen = './data/features/ts_listen.train.csv'
categorical_features_time = ['ts_listen_dayofweek', 
                            'ts_listen_hour']

test_ts_listen = pd.read_csv(path_to_test_ts_listen, encoding='utf-8', usecols=categorical_features_time)
train_ts_listen = pd.read_csv(path_to_train_ts_listen, encoding='utf-8', usecols=categorical_features_time)

test = pd.concat([test, test_ts_listen], axis=1)
train = pd.concat([train, train_ts_listen], axis=1)

df = pd.concat([train, test], ignore_index=True)
df.tail()

,genre_id,ts_listen,album_id,context_type,platform_name,listen_type,user_id,artist_id,is_listened,ts_listen_dayofweek,ts_listen_hour
7578747,0,1479664431,14226364,5,0,1,12641,9285528,0,6,17
7578748,0,1480358099,14226364,1,0,1,10055,9285528,0,0,18
7578749,0,1480616853,14581358,1,0,1,1029,129,0,3,18
7578750,0,1479644510,14581358,1,0,1,4630,129,0,6,12
7578751,0,1480491885,14581358,1,1,1,6467,129,0,2,7


In [4]:
def mean_cum_by_category_by_user(df, category):
    df.sort_values(by=['user_id', 'ts_listen', category], axis=0, inplace=True)
    
    is_listened_cumsum = df.groupby(['user_id',category]).agg({'is_listened':'cumsum'})['is_listened']
    
    cumcount = df.groupby(['user_id',category]).cumcount() + 1
    
    column_name = category+"_cumulative_mean"
    
    df[column_name] = (is_listened_cumsum/cumcount).shift()
    
    fi = df.groupby(['user_id',category], as_index=False).apply(lambda x: x.index[0])
    
    df.ix[fi, column_name] = 0
    
    return column_name

In [5]:
new_cols = [mean_cum_by_category_by_user(df, category) for category in categorical_features+categorical_features_time]
df.head()

,genre_id,ts_listen,album_id,context_type,platform_name,listen_type,user_id,artist_id,is_listened,ts_listen_dayofweek,ts_listen_hour,listen_type_cumulative_mean,genre_id_cumulative_mean,album_id_cumulative_mean,context_type_cumulative_mean,platform_name_cumulative_mean,artist_id_cumulative_mean,ts_listen_dayofweek_cumulative_mean,ts_listen_hour_cumulative_mean
682493,0,1477939775,12811130,0,0,0,0,7508018,1,0,18,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
4454939,0,1477982531,12920172,0,0,0,0,1035779,0,1,6,1.000000,1.000000,0.0,1.000000,1.000000,0.0,0.000000,0.000000
5354523,0,1477982561,13947748,0,0,0,0,4331004,0,1,6,0.500000,0.500000,0.0,0.500000,0.500000,0.0,0.000000,0.000000
5899458,0,1477982582,13976722,0,0,0,0,3469,1,1,6,0.333333,0.333333,0.0,0.333333,0.333333,0.0,0.000000,0.000000
4716574,0,1477982792,13640610,0,0,0,0,282118,1,1,6,0.500000,0.500000,0.0,0.500000,0.500000,0.0,0.333333,0.333333


In [6]:
df.sort_index(inplace=True)
df.tail()

,genre_id,ts_listen,album_id,context_type,platform_name,listen_type,user_id,artist_id,is_listened,ts_listen_dayofweek,ts_listen_hour,listen_type_cumulative_mean,genre_id_cumulative_mean,album_id_cumulative_mean,context_type_cumulative_mean,platform_name_cumulative_mean,artist_id_cumulative_mean,ts_listen_dayofweek_cumulative_mean,ts_listen_hour_cumulative_mean
7578747,0,1479664431,14226364,5,0,1,12641,9285528,0,6,17,0.513514,0.176471,0.0,0.000000,0.208696,0.0,0.200000,0.428571
7578748,0,1480358099,14226364,1,0,1,10055,9285528,0,0,18,0.750000,1.000000,1.0,1.000000,0.967742,1.0,1.000000,0.970588
7578749,0,1480616853,14581358,1,0,1,1029,129,0,3,18,0.822097,0.911032,0.0,0.822097,0.875566,0.0,0.843902,0.950000
7578750,0,1479644510,14581358,1,0,1,4630,129,0,6,12,0.904372,0.900000,0.0,0.840580,0.846914,0.0,0.880952,0.878049
7578751,0,1480491885,14581358,1,1,1,6467,129,0,2,7,0.932203,0.887324,0.0,0.932203,0.925532,0.0,0.928571,0.925926


In [7]:
path_to_test_cum_mean = './data/features/cum_mean.test.csv'
path_to_train_cum_mean = './data/features/cum_mean.train.csv'

test_cum_mean = df.iloc[range(len(train), len(train)+len(test))][new_cols]
test_cum_mean.to_csv(path_to_test_cum_mean, encoding='utf-8', index=False)

train_cum_mean = df.iloc[range(len(train))][new_cols]
train_cum_mean.to_csv(path_to_train_cum_mean, encoding='utf-8', index=False)